In [18]:
# API key -> RGB1NPYOCVMB3Q7J

import requests
import psycopg2
import numpy as np
import psycopg2.extras as extras
import pandas as pd
from datetime import datetime
import pytz
import os

# In listing_status.xlsx Alpha Vantage has provided a list of the ticker symbols available through their API. We can use this list to sort into a list and then download the necessary data for each stock. 

portfolio_stocks = pd.read_excel('/Users/asger/Documents/GitHub/Deep_Learning_Techniques/listing_status.xlsx')['symbol'].tolist()
dft = pd.DataFrame()

for stock in portfolio_stocks:
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={stock}&apikey=RGB1NPYOCVMB3Q7J' # URL for API request, using the stock variable to insert the ticker symbol. 
    r = requests.get(url)
    data = r.json()
    symbol = stock 

    # Transform time series into DataFrame
    # For daily data, the key in the JSON response to access the time series data is 'Time Series (Daily)'
    df = pd.DataFrame(data['Time Series (Daily)']).T.reset_index()

    # Rename columns to reflect their meaning and convert types as necessary
    df = df.rename(columns={
        'index': 'date',  
        '1. open': 'open', 
        '2. high': 'high', 
        '3. low': 'low', 
        '4. close': 'close', 
        '5. volume': 'volume'
    })

    # Convert columns to appropriate data types
    df = df.astype({
        'open': 'float', 
        'high': 'float', 
        'low': 'float', 
        'close': 'float', 
        'volume': 'int'
    })

    # Add a column for the ticker symbol
    df['ticker'] = symbol

    # Concatenate the new DataFrame to the master DataFrame
    dft = pd.concat([dft, df], axis=0)
    
def execute_values(conn, df, table):
    # get new york time zone for log file
    logtime = datetime.now(pytz.timezone('America/New_York'))
    # transform timestamp into unique log file name
    logname = str(logtime)[:19].translate(str.maketrans({'-':'', ' ':'', ':':''}))

    tuples = [tuple(x) for x in df.to_numpy()]

    cols = ','.join(list(df.columns))
    # UPSERT query with conflict handling to avoid duplication
    query = "INSERT INTO %s(%s) VALUES %%s ON CONFLICT DO NOTHING" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
        message = 'PostgreSQL 14 - Stocks DB - Stocks Table has been successfule updated at %s' % logtime
    except (Exception, psycopg2.DatabaseError) as error:
        message = 'Error: %s' % error
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()
    
    # Check if the logs directory exists, if not create it
    if not os.path.exists('logs'):
        os.makedirs('logs')
        
    # write log
    with open(f'logs/{logname}.txt', 'w') as log:
        log.write(f'{logtime}\n{message}')

conn = psycopg2.connect(user="postgres",
                            password="2569",
                            host='localhost',
                            port="4321",
                            database="stocks")

# execute the function for the specific ticker                             
execute_values(conn, dft, 'stocks')

KeyError: 'Time Series (Daily)'